In [36]:
import sys
sys.path.append('/Users/sdbykov/work/self-supervised-halos/')

In [37]:
import requests
import numpy as np
import h5py
import time
from tqdm import tqdm
import os
from glob import glob
import pandas as pd

pd.set_option('display.max_columns', 500)

%matplotlib inline

from utils.data.tng_query import get, HaloInfo
from utils.utils import data_path
data_path = data_path+'tng/'

In [38]:
#dataframe is obrained on the TNG's Jupyter Lab with the code presented below

subhalos_df = pd.read_pickle(data_path[:-4]+'subhalos_df.pkl')
count = len(subhalos_df)

subhalos_df.sample(5)

,SubhaloCM,SubhaloGrNr,SubhaloHalfmassRad,SubhaloHalfmassRadType,SubhaloIDMostbound,SubhaloLen,SubhaloLenType,SubhaloMass,SubhaloMassInHalfRad,SubhaloMassInHalfRadType,SubhaloMassInMaxRad,SubhaloMassInMaxRadType,SubhaloMassInRad,SubhaloMassInRadType,SubhaloMassType,SubhaloParent,SubhaloPos,SubhaloSpin,SubhaloVel,SubhaloVelDisp,SubhaloVmax,SubhaloVmaxRad
subhaloID,,,,,,,,,,,,,,,,,,,,,,
0,"[825.4651, 26360.812, 18043.373]",0,650.651184,"[0.0, 650.6512, 0.0, 0.0, 0.0, 0.0]",5797961333,47955681,"[0, 47955681, 0, 0, 0, 0]",28771.916016,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",8204.043945,"[0.0, 8204.044, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 28771.916, 0.0, 0.0, 0.0, 0.0]",0,"[856.0168, 26345.18, 18318.504]","[14445.006, 911.0099, -29002.348]","[26.606678, -237.90837, -74.87299]",635.132324,981.919495,366.044250
24681,"[19805.244, 47818.293, 48904.41]",1,736.517456,"[0.0, 736.51746, 0.0, 0.0, 0.0, 0.0]",2325180566,45435206,"[0, 45435206, 0, 0, 0, 0]",27259.708984,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",19037.261719,"[0.0, 19037.262, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 27259.709, 0.0, 0.0, 0.0, 0.0]",0,"[19696.723, 47432.613, 48919.953]","[2429.1704, -16896.656, -18658.27]","[-179.63759, 64.59025, -5.3692527]",625.554626,915.306152,977.527710
44062,"[23815.764, 33120.215, 41404.504]",2,402.670837,"[0.0, 402.67084, 0.0, 0.0, 0.0, 0.0]",1987826536,23858229,"[0, 23858229, 0, 0, 0, 0]",14314.195312,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",5323.934082,"[0.0, 5323.934, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 14314.195, 0.0, 0.0, 0.0, 0.0]",0,"[23862.78, 33081.965, 41393.355]","[1515.6438, -6503.139, 34774.26]","[-146.78537, 302.1386, 163.79387]",531.998596,882.996277,293.746246
59551,"[10984.191, 50821.13, 47625.277]",3,541.652344,"[0.0, 541.65234, 0.0, 0.0, 0.0, 0.0]",864268477,38959666,"[0, 38959666, 0, 0, 0, 0]",23374.587891,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",12456.947266,"[0.0, 12456.947, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 23374.588, 0.0, 0.0, 0.0, 0.0]",0,"[10904.087, 50859.2, 47598.504]","[11352.895, -1797.9346, -55533.15]","[281.56152, -93.37164, 79.82162]",594.152649,963.913086,576.757507
74363,"[68437.766, 59068.484, 54036.723]",4,465.475220,"[0.0, 465.47522, 0.0, 0.0, 0.0, 0.0]",4983576378,23558059,"[0, 23558059, 0, 0, 0, 0]",14134.102539,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",4987.670410,"[0.0, 4987.6704, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 14134.103, 0.0, 0.0, 0.0, 0.0]",0,"[68559.01, 59030.53, 53987.703]","[-8654.632, -21578.49, 18774.68]","[158.13545, 67.56957, 182.22719]",490.611969,826.877075,313.814667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958184,"[19628.703, 73197.56, 32715.139]",18249,23.170107,"[0.0, 23.170107, 0.0, 0.0, 0.0, 0.0]",1326341395,11817,"[0, 11817, 0, 0, 0, 0]",7.089832,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",1.758509,"[0.0, 1.7585088, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 7.0898323, 0.0, 0.0, 0.0, 0.0]",0,"[19629.096, 73196.43, 32714.293]","[87.18905, -26.75527, 1.2936268]","[57.95097, 60.15132, -17.731844]",46.138691,86.096901,10.208807
958375,"[4825.373, 24298.955, 16737.156]",18280,24.481579,"[0.0, 24.481579, 0.0, 0.0, 0.0, 0.0]",755109533,11755,"[0, 11755, 0, 0, 0, 0]",7.052634,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",1.277334,"[0.0, 1.2773337, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 7.0526342, 0.0, 0.0, 0.0, 0.0]",0,"[4823.0967, 24299.318, 16737.174]","[-19.911167, -48.11586, 98.87328]","[-605.6193, -90.92572, 197.75418]",45.465691,85.824394,7.463528
958424,"[23199.18, 12566.774, 15933.126]",18291,20.256176,"[0.0, 20.256176, 0.0, 0.0, 0.0, 0.0]",2504767413,11768,"[0, 11768, 0, 0, 0, 0]",7.060434,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",1.636715,"[0.0, 1.636715, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 7.060434, 0.0, 0.0, 0.0, 0.0]",0,"[23198.848, 12567.037, 

```python
# first convert log solar masses into group catalog units
mass_min = 10**11 / 1e10 * 0.704
mass_max = 10**15 / 1e10 * 0.704

# form the search_query string by hand for once
search_query = "?mass__gt=" + str(mass_min) + "&mass__lt=" + str(mass_max) +'&primary_flag__gt=0.9'+'&len__gr=150'

url = 'http://www.tng-project.org/api/TNG100-1-Dark/snapshots/z=0.0/subhalos/' + search_query
subhalos_all = get(url, {'limit':20000})

count = subhalos_all['count']
print(f"{count} halos with search criteria \n \t {search_query}")

```

In [40]:
for id in tqdm(subhalos_df.index.values[::-1]):
    halo_info = HaloInfo(id)
    halo_info.download_halo_snapshot(retry=False, verbose=False)
    time.sleep(0.01)
    

 20%|█▉        | 3304/16544 [4:00:41<16:04:30,  4.37s/it]


KeyboardInterrupt: 

In [34]:
#mass history is prepared acording to the code below on the jupyter lab of TNG
import pickle
subhalos_history = pickle.load(open(data_path[:-4]+'subhalos_history.pkl', 'rb'))

```python
fields = ['SubhaloMass','SubfindID','SnapNum']


def get_merger_history(subhaloID):
    tree = il.sublink.loadTree(base_path, snapshot, subhaloID, fields=fields, onlyMPB=True)
    mass_msun = tree['SubhaloMass'] * 1e10 / 0.6774
    snap_num = tree['SnapNum']
    
    return snap_num, mass_msun

results = {}

for i, halo_id in enumerate(subhalos_df.index.values):
    if i%100==0: print(f'finished {i}/{count}')
    
    snap_num, mass_msun = get_merger_history(halo_id)
    
    results[halo_id] = {'snap':snap_num,'mass':mass_msun}

```

---

## Acces to snapshot of a subhalo

```python
### loading subhalo cutout (relatively long process!)
import illustris_python as il
import h5py

snapshot = 99 
#sim = '../sims.TNG/TNG100-1-Dark/' #for tng lab
sim = '/virgotng/universe/IllustrisTNG/TNG100-1-Dark/' #for freya
base_path = sim+'output/'
subhaloID = 59551

snap = il.snapshot.loadSubhalo(base_path, snapNum=snapshot, id = subhaloID, partType = 1)
snap

with h5py.File('TMP_subhalo_cutout.hdf5', 'w') as f:
    #save coordinates and gravitational potential
    f.create_dataset('Coordinates', data=snap['Coordinates'])
    f.create_dataset('Potential', data=snap['Potential'])

```